In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.optimizers import Adam

# Note: Make sure to replace '/path/to/save/model' and '/path/to/norm_params.csv' with actual paths where you want to save your model and normalization parameters.

In [ ]:
# Load and combine datasets
dataset_path = "datasets/nba-shot-chart-dataset-2000-2024"
file_paths = ['/path/to/20001031.csv', '/path/to/20001101.csv', '/path/to/20001102.csv']
dfs = [pd.read_csv(file_path) for file_path in file_paths]
combined_df = pd.concat(dfs, ignore_index=True)

In [ ]:
# Data cleaning and preprocessing
cleaned_df = combined_df.dropna(subset=['made', 'shotX', 'shotY'])
cleaned_df['shotX'] = (cleaned_df['shotX'] - cleaned_df['shotX'].mean()) / cleaned_df['shotX'].std()
cleaned_df['shotY'] = (cleaned_df['shotY'] - cleaned_df['shotY'].mean()) / cleaned_df['shotY'].std()
cleaned_df['made'] = cleaned_df['made'].astype(int)


In [ ]:
# Save normalization parameters for use in prediction script
x_mean = cleaned_df['shotX'].mean()
x_std = cleaned_df['shotX'].std()
y_mean = cleaned_df['shotY'].mean()
y_std = cleaned_df['shotY'].std()
norm_params = {'x_mean': x_mean, 'x_std': x_std, 'y_mean': y_mean, 'y_std': y_std}
pd.DataFrame(norm_params, index=[0]).to_csv('/path/to/norm_params.csv', index=False)


In [ ]:
# Preparing data for model
X = cleaned_df[['shotX', 'shotY']]
y = cleaned_df['made']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Model definition
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Model compilation
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Model training
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

In [ ]:
# Model evaluation
evaluation = model.evaluate(X_test, y_test)
print(f'Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}')

# Save the model for future use
model.save('model/ann_model')
